In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Normalization
from tensorflow.keras.models import Sequential, Model

In [2]:
print(tf.__version__)

2.8.0


In [3]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

In [4]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

Read the CSV file using pandas

In [5]:
df = pd.read_csv(csv_file)

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


If label data is not devided in data, devide label data and train data

In [7]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [8]:
target = df.pop('target')

###### distinguish numeric data which is not categorical
age, trestbps, chol, thalach, oldpeak

In [9]:
for name, values in df.items():
    print(name)
    print(df[name].unique())
    print(df[name].nunique())
    print('==================================================================================')

age
[63 67 37 41 56 62 57 53 44 52 48 54 49 64 58 60 50 66 43 40 69 59 42 55
 61 65 51 45 39 68 47 35 29 70 46 77 38 34 74 76 71]
41
sex
[1 0]
2
cp
[1 4 3 2 0]
5
trestbps
[145 160 120 130 140 172 150 110 132 117 135 155 125 104 180 138 108 128
 100 200 124  94 122 170 105 165 112 102 152 115 118 101 126 142 174 134
 148 178 158 192 129 144 123 136 146 106 156 154 114 164]
50
chol
[233 286 229 250 204 236 268 354 254 203 192 294 256 263 199 168 239 275
 266 211 283 284 224 206 219 340 226 247 167 230 335 234 177 276 353 243
 225 269 267 248 197 360 258 308 245 270 208 264 321 274 325 235 257 302
 164 231 141 252 255 183 330 222 217 282 288 220 209 227 261 213 174 281
 198 221 205 309 240 289 318 298 265 564 246 322 299 300 293 277 304 214
 207 160 249 394 212 184 315 409 244 305 195 196 273 126 313 259 200 262
 215 228 193 303 271 210 327 149 201 295 306 178 237 218 223 242 319 166
 180 311 278 232 253 342 169 187 157 176 241 131 175 417 290 172 216 188
 185 326 260 182 307 186 341 407]

In [10]:
numeric_features_names = ['age', 'thalach', 'trestbps', 'chol', 'oldpeak']
numeric_features = df[numeric_features_names]
numeric_features.head()

,age,thalach,trestbps,chol,oldpeak
0,63,150,145,233,2.3
1,67,108,160,286,1.5
2,67,129,120,229,2.6
3,37,187,130,250,3.5
4,41,172,130,204,1.4


In [11]:
 tf.convert_to_tensor(numeric_features)

Metal device set to: Apple M1


2022-07-30 11:35:43.733940: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-30 11:35:43.734108: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(303, 5), dtype=float64, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]])>

##### With Model fit

In [12]:
normalizer = Normalization(axis=-1)
normalizer.adapt(numeric_features)
normalizer(numeric_features.iloc[:3])

2022-07-30 11:35:43.788124: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-30 11:35:43.867753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 11:35:43.891013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.933839  ,  0.03480718,  0.7457807 , -0.26008663,  1.0680454 ],
       [ 1.3782104 , -1.7806164 ,  1.5923283 ,  0.75738764,  0.38022867],
       [ 1.3782104 , -0.87290466, -0.66513205, -0.33687714,  1.3259766 ]],
      dtype=float32)>

In [13]:
normalizer_axis0 = Normalization(axis=0)
normalizer_axis0.adapt(numeric_features)

# ValueError: All `axis` values to be kept must have known shape. 
# Got axis: (0,), input shape: [None, 5], with unknown axis at index: 0

# why is input shape None........
# numeric_features shape is [303, 5], 
# tf.convert_to_tensor(numeric_features) shape is [303, 5]

ValueError: in user code:

    File "/Users/rainyseason/miniforge3/envs/tf25/lib/python3.8/site-packages/keras/engine/base_preprocessing_layer.py", line 117, in adapt_step  *
        self._adapt_maybe_build(data)
    File "/Users/rainyseason/miniforge3/envs/tf25/lib/python3.8/site-packages/keras/engine/base_preprocessing_layer.py", line 285, in _adapt_maybe_build  **
        self.build(data_shape)
    File "/Users/rainyseason/miniforge3/envs/tf25/lib/python3.8/site-packages/keras/layers/preprocessing/normalization.py", line 150, in build
        raise ValueError(

    ValueError: All `axis` values to be kept must have known shape. Got axis: (0,), input shape: [None, 5], with unknown axis at index: 0


In [15]:
a = [[ 0.933839  ,  0.03480718,  0.7457807 , -0.26008663,  1.0680454 ]]

In [16]:
normalizer_dogs = Normalization(axis=-1)
normalizer_dogs.adapt(a)
normalizer_dogs(numeric_features.iloc[:3])

# ValueError: All `axis` values to be kept must have known shape. 
# Got axis: (0,), input shape: [None, 5], with unknown axis at index: 0

# why is input shape None........
# numeric_features shape is [303, 5], 
# tf.convert_to_tensor(numeric_features) shape is [303, 5]

2022-07-30 11:37:19.275567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 11:37:19.298216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[6.2066163e+08, 1.4996520e+09, 1.4425422e+09, 2.3326008e+09,
        1.2319546e+07],
       [6.6066163e+08, 1.0796520e+09, 1.5925422e+09, 2.8626010e+09,
        4.3195460e+06],
       [6.6066163e+08, 1.2896520e+09, 1.1925422e+09, 2.2926008e+09,
        1.5319545e+07]], dtype=float32)>

In [17]:
def get_basic_model():
    model = Sequential([
        normalizer,
        Dense(10, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    return model

def get_dogs_model():
    model = Sequential([
        normalizer_dogs,
        Dense(10, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    return model

In [18]:
model = get_basic_model()
model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
  1/152 [..............................] - ETA: 1:12 - loss: 0.7028 - accuracy: 0.5000

2022-07-30 11:37:20.262369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 2s 8ms/step - loss: 0.7046 - accuracy: 0.7063
Epoch 2/15
152/152 [==============================] - 1s 7ms/step - loss: 0.6225 - accuracy: 0.7360
Epoch 3/15
152/152 [==============================] - 1s 7ms/step - loss: 0.5346 - accuracy: 0.7525
Epoch 4/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4798 - accuracy: 0.7492
Epoch 5/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4516 - accuracy: 0.7789
Epoch 6/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4384 - accuracy: 0.7855
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4310 - accuracy: 0.7855
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4271 - accuracy: 0.7888
Epoch 9/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4239 - accuracy: 0.7921
Epoch 10/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4209 - accuracy: 0.7822
Epoch 11/1

In [19]:
model_dogs = get_dogs_model()
model_dogs.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
  8/152 [>.............................] - ETA: 1s - loss: 276997312.0000 - accuracy: 0.2500  

2022-07-30 11:37:37.065188: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 2s 7ms/step - loss: 42215180.0000 - accuracy: 0.5545
Epoch 2/15
152/152 [==============================] - 1s 7ms/step - loss: 8962341.0000 - accuracy: 0.5974
Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 7467221.0000 - accuracy: 0.6667
Epoch 4/15
152/152 [==============================] - 1s 7ms/step - loss: 4745821.5000 - accuracy: 0.6865
Epoch 5/15
152/152 [==============================] - 1s 7ms/step - loss: 5446935.5000 - accuracy: 0.6997
Epoch 6/15
152/152 [==============================] - 1s 7ms/step - loss: 3377617.2500 - accuracy: 0.7162
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 4253352.5000 - accuracy: 0.6766
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 3987580.0000 - accuracy: 0.7195
Epoch 9/15
152/152 [==============================] - 1s 7ms/step - loss: 4644278.0000 - accuracy: 0.6931
Epoch 10/15
152/152 [==============================] - 1

##### With tf.data

In [20]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 63. , 150. , 145. , 233. ,   2.3])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 108. , 160. , 286. ,   1.5])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 67. , 129. , 120. , 229. ,   2.6])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [21]:
numeric_batchs = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
model.fit(numeric_batchs, epochs=15)

Epoch 1/15
  8/152 [>.............................] - ETA: 1s - loss: 0.7796 - accuracy: 0.5625 

2022-07-30 11:37:53.716259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 2s 8ms/step - loss: 0.6594 - accuracy: 0.7591
Epoch 2/15
152/152 [==============================] - 1s 7ms/step - loss: 0.5538 - accuracy: 0.7591
Epoch 3/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4966 - accuracy: 0.7789
Epoch 4/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4627 - accuracy: 0.7822
Epoch 5/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4462 - accuracy: 0.7789
Epoch 6/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4360 - accuracy: 0.7789
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4296 - accuracy: 0.7888
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4291 - accuracy: 0.7789
Epoch 9/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4255 - accuracy: 0.7888
Epoch 10/15
152/152 [==============================] - 1s 7ms/step - loss: 0.4234 - accuracy: 0.7855
Epoch 11/1

In [22]:
model_dogs = get_dogs_model()
model_dogs.fit(numeric_batchs, epochs=15)

Epoch 1/15
  8/152 [>.............................] - ETA: 1s - loss: 313006752.0000 - accuracy: 0.2500     

2022-07-30 11:38:10.158580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 1s 7ms/step - loss: 45198988.0000 - accuracy: 0.5677
Epoch 2/15
152/152 [==============================] - 1s 7ms/step - loss: 9842363.0000 - accuracy: 0.6172
Epoch 3/15
152/152 [==============================] - 1s 7ms/step - loss: 6902947.5000 - accuracy: 0.6568
Epoch 4/15
152/152 [==============================] - 1s 7ms/step - loss: 5347527.0000 - accuracy: 0.6898
Epoch 5/15
152/152 [==============================] - 1s 7ms/step - loss: 4963303.0000 - accuracy: 0.6568
Epoch 6/15
152/152 [==============================] - 1s 7ms/step - loss: 4342128.0000 - accuracy: 0.7327
Epoch 7/15
152/152 [==============================] - 1s 7ms/step - loss: 3238033.5000 - accuracy: 0.6931
Epoch 8/15
152/152 [==============================] - 1s 7ms/step - loss: 5117220.0000 - accuracy: 0.7030
Epoch 9/15
152/152 [==============================] - 1s 7ms/step - loss: 4121550.7500 - accuracy: 0.7063
Epoch 10/15
152/152 [==============================] - 1

##### A DataFrame as a dictionary

In [23]:
dict_numeric_features = dict(numeric_features)

In [24]:
dict_numeric_features

{'age': 0      63
 1      67
 2      67
 3      37
 4      41
        ..
 298    52
 299    43
 300    65
 301    48
 302    63
 Name: age, Length: 303, dtype: int64,
 'thalach': 0      150
 1      108
 2      129
 3      187
 4      172
       ... 
 298    190
 299    136
 300    127
 301    150
 302    154
 Name: thalach, Length: 303, dtype: int64,
 'trestbps': 0      145
 1      160
 2      120
 3      130
 4      130
       ... 
 298    118
 299    132
 300    135
 301    130
 302    150
 Name: trestbps, Length: 303, dtype: int64,
 'chol': 0      233
 1      286
 2      229
 3      250
 4      204
       ... 
 298    186
 299    341
 300    254
 301    256
 302    407
 Name: chol, Length: 303, dtype: int64,
 'oldpeak': 0      2.3
 1      1.5
 2      2.6
 3      3.5
 4      1.4
       ... 
 298    0.0
 299    3.0
 300    2.8
 301    0.0
 302    4.0
 Name: oldpeak, Length: 303, dtype: float64}

In [25]:
numeric_dict_ds = tf.data.Dataset.from_tensor_slices((dict_numeric_features, target))

In [31]:
for row in numeric_dict_ds.take(3):
    print(row)

({'age': <tf.Tensor: shape=(), dtype=int64, numpy=63>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=150>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=145>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=233>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.3>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=108>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=160>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=286>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=1.5>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
({'age': <tf.Tensor: shape=(), dtype=int64, numpy=67>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=129>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=120>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=229>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=2.6>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


###### Dictionaries with Keras
1. The model-subclass style

In [38]:
temp = tf.cast(dict_numeric_features['age'], dtype=tf.float32)

In [43]:
temp

<tf.Tensor: shape=(303,), dtype=float32, numpy=
array([63., 67., 67., 37., 41., 56., 62., 57., 63., 53., 57., 56., 56.,
       44., 52., 57., 48., 54., 48., 49., 64., 58., 58., 58., 60., 50.,
       58., 66., 43., 40., 69., 60., 64., 59., 44., 42., 43., 57., 55.,
       61., 65., 65., 67., 62., 65., 44., 65., 60., 51., 48., 58., 45.,
       53., 39., 68., 52., 44., 47., 53., 51., 66., 62., 62., 44., 63.,
       52., 62., 41., 58., 35., 63., 51., 55., 65., 45., 56., 54., 44.,
       62., 54., 51., 29., 51., 43., 55., 70., 62., 35., 51., 59., 59.,
       52., 64., 58., 47., 57., 41., 45., 60., 52., 42., 67., 55., 64.,
       70., 51., 58., 60., 68., 46., 77., 54., 58., 48., 57., 54., 35.,
       45., 70., 53., 59., 62., 64., 57., 52., 56., 43., 53., 48., 56.,
       42., 59., 60., 63., 42., 66., 54., 69., 50., 51., 62., 68., 67.,
       69., 45., 50., 59., 50., 64., 57., 64., 43., 45., 58., 50., 55.,
       62., 37., 38., 41., 66., 52., 56., 46., 46., 64., 59., 41., 54.,
       39., 53.,

In [51]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in inputs.keys():
        values.append(tf.cast(inputs[key], dtype=tf.float32))
    
    return fun(values, axis=-1)

In [52]:
temp = stack_dict(dict_numeric_features)
for i in temp:
    print(i)

tf.Tensor([ 63.  150.  145.  233.    2.3], shape=(5,), dtype=float32)
tf.Tensor([ 67.  108.  160.  286.    1.5], shape=(5,), dtype=float32)
tf.Tensor([ 67.  129.  120.  229.    2.6], shape=(5,), dtype=float32)
tf.Tensor([ 37.  187.  130.  250.    3.5], shape=(5,), dtype=float32)
tf.Tensor([ 41.  172.  130.  204.    1.4], shape=(5,), dtype=float32)
tf.Tensor([ 56.  178.  120.  236.    0.8], shape=(5,), dtype=float32)
tf.Tensor([ 62.  160.  140.  268.    3.6], shape=(5,), dtype=float32)
tf.Tensor([ 57.  163.  120.  354.    0.6], shape=(5,), dtype=float32)
tf.Tensor([ 63.  147.  130.  254.    1.4], shape=(5,), dtype=float32)
tf.Tensor([ 53.  155.  140.  203.    3.1], shape=(5,), dtype=float32)
tf.Tensor([ 57.  148.  140.  192.    0.4], shape=(5,), dtype=float32)
tf.Tensor([ 56.  153.  140.  294.    1.3], shape=(5,), dtype=float32)
tf.Tensor([ 56.  142.  130.  256.    0.6], shape=(5,), dtype=float32)
tf.Tensor([ 44. 173. 120. 263.   0.], shape=(5,), dtype=float32)
tf.Tensor([ 52.  162.  17

tf.Tensor([ 39. 179.  94. 199.   0.], shape=(5,), dtype=float32)
tf.Tensor([ 53.  95. 123. 282.   2.], shape=(5,), dtype=float32)
tf.Tensor([ 63.  169.  108.  269.    1.8], shape=(5,), dtype=float32)
tf.Tensor([ 34.  192.  118.  210.    0.7], shape=(5,), dtype=float32)
tf.Tensor([4.70e+01 1.43e+02 1.12e+02 2.04e+02 1.00e-01], shape=(5,), dtype=float32)
tf.Tensor([ 67. 172. 152. 277.   0.], shape=(5,), dtype=float32)
tf.Tensor([ 54. 108. 110. 206.   0.], shape=(5,), dtype=float32)
tf.Tensor([6.60e+01 1.32e+02 1.12e+02 2.12e+02 1.00e-01], shape=(5,), dtype=float32)
tf.Tensor([5.20e+01 1.69e+02 1.36e+02 1.96e+02 1.00e-01], shape=(5,), dtype=float32)
tf.Tensor([ 55.  117.  180.  327.    3.4], shape=(5,), dtype=float32)
tf.Tensor([ 49.  126.  118.  149.    0.8], shape=(5,), dtype=float32)
tf.Tensor([7.40e+01 1.21e+02 1.20e+02 2.69e+02 2.00e-01], shape=(5,), dtype=float32)
tf.Tensor([ 54. 163. 160. 201.   0.], shape=(5,), dtype=float32)
tf.Tensor([ 54.  116.  122.  286.    3.2], shape=(5,), 

In [53]:
temp

<tf.Tensor: shape=(303, 5), dtype=float32, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]], dtype=float32)>

In [54]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__(self)
        
        self.normalizer = Normalization(axis=-1)
        
        self.seq = Sequential([
            self.normalizer,
            Dense(10, activation='relu'),
            Dense(10, activation='relu'),
            Dense(1)
        ])
        
    def adapt(self, inputs):
        inputs = stack_dict(inputs)
        self.normalizer.adapt(inputs)
        
    def call(self, inputs):
        inputs = stack_dict(inputs)
        result = self.seq(inputs)
        
        return result

In [56]:
model_general = MyModel()
model_general.adapt(dict_numeric_features)
model_general.compile(optimizer='adam',
                     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                     metrics=['accuracy'],
                     run_eagerly=True)

2022-07-30 12:07:48.656162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 12:07:48.678578: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [69]:
dict_numeric_one_line = dict(numeric_features.iloc[:1])

dict_numeric_one_line

{'age': 0    63
 Name: age, dtype: int64,
 'thalach': 0    150
 Name: thalach, dtype: int64,
 'trestbps': 0    145
 Name: trestbps, dtype: int64,
 'chol': 0    233
 Name: chol, dtype: int64,
 'oldpeak': 0    2.3
 Name: oldpeak, dtype: float64}

In [70]:
model_dogs = MyModel()
model_dogs.adapt(dict_numeric_one_line)
model_dogs.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'],
                  run_eagerly=True)

2022-07-30 12:15:13.084610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-30 12:15:13.106782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [72]:
model_general.fit(dict_numeric_features, target, epochs=5, batch_size=BATCH_SIZE)

Epoch 1/5
152/152 [==============================] - 4s 23ms/step - loss: 0.6570 - accuracy: 0.7723
Epoch 2/5
152/152 [==============================] - 3s 21ms/step - loss: 0.5552 - accuracy: 0.7690
Epoch 3/5
152/152 [==============================] - 3s 21ms/step - loss: 0.4991 - accuracy: 0.7690
Epoch 4/5
152/152 [==============================] - 3s 22ms/step - loss: 0.4627 - accuracy: 0.7855
Epoch 5/5
152/152 [==============================] - 3s 22ms/step - loss: 0.4431 - accuracy: 0.7888


In [75]:
model_dogs.fit(dict_numeric_features, target, epochs=5, batch_size=BATCH_SIZE)

Epoch 1/5
152/152 [==============================] - 3s 22ms/step - loss: 18971304.0000 - accuracy: 0.4455
Epoch 2/5
152/152 [==============================] - 3s 21ms/step - loss: 4276564.5000 - accuracy: 0.6205
Epoch 3/5
152/152 [==============================] - 3s 22ms/step - loss: 3139886.5000 - accuracy: 0.6370
Epoch 4/5
152/152 [==============================] - 3s 21ms/step - loss: 2368802.7500 - accuracy: 0.6733
Epoch 5/5
152/152 [==============================] - 3s 22ms/step - loss: 2629194.7500 - accuracy: 0.6766


In [83]:
model_general.predict(dict(numeric_features.iloc[:3]))

array([[[-0.19705318]],

       [[ 0.60033095]],

       [[ 0.24553859]]], dtype=float32)

In [85]:
model_dogs.predict(dict(numeric_features.iloc[:3]))

array([[[-3.4557015e-01]],

       [[-1.0302871e+05]],

       [[ 5.4937445e+06]]], dtype=float32)

2. The keras functional style